In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import pickle
import h5py

In [ ]:

total_subjects=['sub1','sub2','sub3','sub4','sub5','sub6','sub7','sub8','sub9','sub10','sub11','sub12','sub13','sub14','sub15']

In [ ]:
strip_grid_no_list={'sub1':[0,1,2,3],
                    'sub2':[0,1,2,3,4,5],
                    'sub3':[0,1,2,3,4],
                    'sub4':[0,1,2,3,4],
                    'sub5':[0,1,2,3],
                    'sub6':[0,1,2,3,4],
                    'sub7':[0,1,2],
                    'sub8':[0,2,3],
                    'sub9':[0,1,2,3],
                    'sub10':[0,1],
                    'sub11':[0,1,2],
                    'sub12':[0,1,2],
                    'sub13':[0,1,2,3],
                    'sub14':[0,1,2,3],
                    'sub15':[0,1,2,3,4]
}
                   

# Among the two lists in badnull_ans_dict, the first contains cases where the answer was stated incorrectly (i.e., the opposite of the correct answer), and the second contains experimental errors—such as failing to give an answer, speaking during the baseline period, or similar issues.
liv_nonliv_badnull_ans_dict = {'sub1': [[], []],
                               'sub2': [[], []],
                               'sub3': [[49], []],
                               'sub4': [[], []],
                               'sub5': [[5, 7], []],
                               'sub6': [[41], [17]],
                               'sub7': [[], []],
                               'sub8': [[], [33]],
                               'sub9' : [[8],[13]],
                               'sub10' : [[],[36]],
                               'sub11' : [[],[]],
                               'sub12' : [[],[]],
                               'sub13' : [[16],[]],
                               'sub14': [[10], [23]],
                               'sub15': [[], []],
                               }


body_nonbody_badnull_ans_dict = {'sub1': [[], []],
                               'sub2': [[], []],
                               'sub3': [[], []],
                               'sub4': [[], [3,7]],
                               'sub5': [[], []],
                               'sub6': [[], []],
                               'sub7': [[], []],
                               'sub8': [[], []],
                               'sub9' : [[],[]],
                               'sub10' : [[],[4]],
                               'sub11' : [[],[]],
                               'sub12' : [[],[]],
                               'sub13' : [[],[]],
                               'sub14': [[], [0]],
                               'sub15': [[], []],
                               }


body_nonbody_badnull_ans_dict = {'sub1': [[], []],
                                 'sub2': [[], []],
                                 'sub3': [[], []],
                                 'sub4': [[], [3+72, 7+72]],
                                 'sub5': [[], []],
                                 'sub6': [[], []],
                                 'sub7': [[], []],
                                 'sub8': [[], []],
                                 'sub9' : [[],[]],
                                 'sub10' : [[i for i in range(54,72)],[4+72]+[i for i in range(126,144)]],
                                 'sub11' : [[],[]],
                                 'sub12' : [[],[]],
                                 'sub13' : [[],[]],
                                 'sub14': [[], [0+72]],
                                 'sub15': [[], []],
                                 }


In [ ]:
# add 72 in bodypart
def bad_trial_giver(pat_name, strip_grid_no_list):
    total_badtrial=[[],[],[],[],[],[],[]]
    ch_info_dirs = 'data_path/'+pat_name+'/'+pat_name+'good_ch.mat'
    with h5py.File(ch_info_dirs, 'r') as file:
        ch_info_raw = file['good_ch_index'][:]
    max_grid_group = int(ch_info_raw[-1]//1000)
    ch_info = [[] for i in range(max_grid_group)]
    for each_ch_info in ch_info_raw:
        grid_group = int(each_ch_info//1000)-1
        grid_num = int(each_ch_info%1000)
        ch_info[grid_group].append(grid_num)
    ch_indexing=[0]
    cnter_ch=0
    strip_grid_no = str(strip_grid_no_list[pat_name])
    file_path = "l_"+pat_name+'_'+strip_grid_no+"_index.pkl"
    with open(file_path, 'rb') as f:
        checkbox_indexs = pickle.load(f)
    print(checkbox_indexs)
    file_path = "badtrials/l_"+pat_name+'_'+strip_grid_no+".pkl"
    with open(file_path, 'rb') as f: #6
        l_badtrial_raw = np.array(pickle.load(f))
    
    file_path = "badtrials/b_"+pat_name+'_'+strip_grid_no+".pkl"
    with open(file_path, 'rb') as f: #7
        b_badtrial_raw = np.array(pickle.load(f))    
    
    for i in ch_info:
        ch_indexing.append(ch_indexing[cnter_ch]+len(i))
        cnter_ch +=1
    
    for idx in range(len(strip_grid_no_list[pat_name])):
        checkbox_index = checkbox_indexs[idx]
        ch_cnt = len(ch_info[strip_grid_no_list[pat_name][idx]])
        for point in range(7):
            check_bullean_l = np.where(np.sum(l_badtrial_raw[checkbox_index,:,:][:,[0,point+1]],axis=1)>0,1,0)
            check_bullean_b = np.where(np.sum(b_badtrial_raw[checkbox_index,:,:][:,[0,point+1]],axis=1)>0,1,0)
            bad_trial_list = []
            for i in range(72):
                if check_bullean_l[i] == 1:
                    bad_trial_list.append(i)
            for i in range(72):
                if check_bullean_b[i] == 1:
                    bad_trial_list.append(i+72)
            for i in range(ch_cnt):
                total_badtrial[point].append(bad_trial_list)
    return total_badtrial
    
    
    

In [ ]:
def getPatgoodch(pat_name,strip_grid_no_list):
    strip_grid_no = strip_grid_no_list[pat_name]
    return_list=[]
    ch_info_dirs = 'data_path/'+pat_name+'/'+pat_name+'good_ch.mat'
    with h5py.File(ch_info_dirs, 'r') as file:
        ch_info_raw = file['good_ch_index'][:]
    max_grid_group = int(ch_info_raw[-1]//1000)
    ch_info = [[] for i in range(max_grid_group)]
    for each_ch_info in ch_info_raw:
        grid_group = int(each_ch_info//1000)-1
        grid_num = int(each_ch_info%1000)
        ch_info[grid_group].append(grid_num)
    ch_indexing=[0]
    cnter_ch=0
    for i in ch_info:
        #print(len(i))
        ch_indexing.append(ch_indexing[cnter_ch]+len(i))
        cnter_ch +=1
    for strip_grid_idx in strip_grid_no:
        for i in range(ch_indexing[strip_grid_idx],ch_indexing[strip_grid_idx+1]):
            return_list.append(i)
    print(ch_indexing)
    print(len(ch_info_raw))
    print(len(return_list))
    return return_list




In [ ]:
#FILE LOAD

q_offsettiming=[3407,4070,3823,4302] #offset time for each sentence

def file_loader_and_npconcate_TFmean(pat_name,cutstart,cutend,ver):
    dirs = 'D:/GwonYonghyeon/data/250102_preprocessed/'+pat_name+'/'+pat_name+'TF_freqenv_ver'+str(ver)
    val_name = "TF_mean_epoch"
    fs_ratio = 2
    with h5py.File(dirs+'sem1.mat', 'r') as file:
        data_sem1 = file[val_name][:]
    data_sem1_copy = data_sem1[:, fs_ratio*cutstart:fs_ratio*cutend, :, :].copy()
    del data_sem1
    with h5py.File(dirs+'sem2.mat', 'r') as file:
        data_sem2 = file[val_name][:]
    data_sem2_copy = data_sem2[:, fs_ratio*cutstart:fs_ratio*cutend, :, :].copy()
    del data_sem2
    with h5py.File(dirs+'sem5.mat', 'r') as file:
        data_sem5 = file[val_name][:]
    data_sem5_copy = data_sem5[:, fs_ratio*cutstart:fs_ratio*cutend, :, :].copy()
    del data_sem5
    with h5py.File(dirs+'sem6.mat', 'r') as file:
        data_sem6 = file[val_name][:]
    data_sem6_copy = data_sem6[:, fs_ratio*cutstart:fs_ratio*cutend, :, :].copy()
    del data_sem6

    liv_nonliv_raw_data = np.concatenate((data_sem1_copy, data_sem2_copy,data_sem5_copy,data_sem6_copy), axis=0)
    print("liv_nonliv_load_complete. || shape : ",   liv_nonliv_raw_data.shape)

    with h5py.File(dirs+'sem3.mat', 'r') as file:
        data_sem3 = file[val_name][:]
    data_sem3_copy = data_sem3[:, fs_ratio*cutstart:fs_ratio*cutend, :, :].copy()
    del data_sem3
    with h5py.File(dirs+'sem4.mat', 'r') as file:
        data_sem4 = file[val_name][:]
    data_sem4_copy = data_sem4[:, fs_ratio*cutstart:fs_ratio*cutend, :, :].copy()
    del data_sem4
    with h5py.File(dirs+'sem7.mat', 'r') as file:
        data_sem7 = file[val_name][:]
    data_sem7_copy = data_sem7[:, fs_ratio*cutstart:fs_ratio*cutend, :, :].copy()
    del data_sem7
    with h5py.File(dirs+'sem8.mat', 'r') as file:
        data_sem8 = file[val_name][:]
    data_sem8_copy = data_sem8[:, fs_ratio*cutstart:fs_ratio*cutend, :, :].copy()
    del data_sem8

    body_nonbody_raw_data = np.concatenate((data_sem3_copy, data_sem4_copy,data_sem7_copy,data_sem8_copy), axis=0)
    print("body_nonbody_load_complete. || shape : ",   body_nonbody_raw_data.shape)
    
    return liv_nonliv_raw_data, body_nonbody_raw_data

def file_loader_and_npconcate_question_TFmean(pat_name,q_offset,liv_nonliv_raw_answer,body_nonbody_raw_answer,ver):
    time_window = 600
    dirs = 'D:/GwonYonghyeon/data/250102_preprocessed/'+pat_name+'/'+pat_name+'TF_freqenv_ver'+str(ver)
    val_name ="TF_mean_epoch"
    fs_ratio = 2 
    with h5py.File(dirs+'sem1.mat', 'r') as file:
        data_sem1 = file[val_name][:]
    data_sem1_copy = np.zeros((data_sem1.shape[0],time_window*fs_ratio,data_sem1.shape[2],data_sem1.shape[3]))
    for i in range(18):
        if liv_nonliv_raw_answer[i] == 0:
            data_sem1_copy[i,:,:,:] = data_sem1[i,fs_ratio*q_offset[0]:fs_ratio*(q_offset[0]+time_window),:,:].copy()
        if liv_nonliv_raw_answer[i] == 1:
            data_sem1_copy[i,:,:,:] = data_sem1[i,fs_ratio*q_offset[1]:fs_ratio*(q_offset[1]+time_window),:,:].copy()
    del data_sem1
    with h5py.File(dirs+'sem2.mat', 'r') as file:
        data_sem2 = file[val_name][:]
    data_sem2_copy = np.zeros((data_sem2.shape[0],time_window*fs_ratio,data_sem2.shape[2],data_sem2.shape[3]))
    for i in range(18):
        if liv_nonliv_raw_answer[i+18] == 0:
            data_sem2_copy[i,:,:,:] = data_sem2[i,fs_ratio*q_offset[0]:fs_ratio*(q_offset[0]+time_window),:,:].copy()
        if liv_nonliv_raw_answer[i+18] == 1:
            data_sem2_copy[i,:,:,:] = data_sem2[i,fs_ratio*q_offset[1]:fs_ratio*(q_offset[1]+time_window),:,:].copy()
    del data_sem2
    with h5py.File(dirs+'sem5.mat', 'r') as file:
        data_sem5 = file[val_name][:]
    data_sem5_copy = np.zeros((data_sem5.shape[0],time_window*fs_ratio,data_sem5.shape[2],data_sem5.shape[3]))
    for i in range(18):
        if liv_nonliv_raw_answer[i+36] == 0:
            data_sem5_copy[i,:,:,:] = data_sem5[i,fs_ratio*q_offset[0]:fs_ratio*(q_offset[0]+time_window),:,:].copy()
        if liv_nonliv_raw_answer[i+36] == 1:
            data_sem5_copy[i,:,:,:] = data_sem5[i,fs_ratio*q_offset[1]:fs_ratio*(q_offset[1]+time_window),:,:].copy()
    del data_sem5
    with h5py.File(dirs+'sem6.mat', 'r') as file:
        data_sem6 = file[val_name][:]
    data_sem6_copy = np.zeros((data_sem6.shape[0],time_window*fs_ratio,data_sem6.shape[2],data_sem6.shape[3]))
    for i in range(18):
        if liv_nonliv_raw_answer[i+54]== 0:
            data_sem6_copy[i,:,:,:] = data_sem6[i,fs_ratio*q_offset[0]:fs_ratio*(q_offset[0]+time_window),:,:].copy()
        if liv_nonliv_raw_answer[i+54] == 1:
            data_sem6_copy[i,:,:,:] = data_sem6[i,fs_ratio*q_offset[1]:fs_ratio*(q_offset[1]+time_window),:,:].copy()
    del data_sem6

    liv_nonliv_raw_data = np.concatenate((data_sem1_copy, data_sem2_copy,data_sem5_copy,data_sem6_copy), axis=0)
    print("liv_nonliv_load_complete. || shape : ",   liv_nonliv_raw_data.shape)

    with h5py.File(dirs+'sem3.mat', 'r') as file:
        data_sem3 = file[val_name][:]
    data_sem3_copy = np.zeros((data_sem3.shape[0],time_window*fs_ratio,data_sem3.shape[2],data_sem3.shape[3]))
    for i in range(18):
        if body_nonbody_raw_answer[i] == 0:
            data_sem3_copy[i,:,:,:] = data_sem3[i,fs_ratio*q_offset[2]:fs_ratio*(q_offset[2]+time_window),:,:].copy()
        if body_nonbody_raw_answer[i] == 1:
            data_sem3_copy[i,:,:,:] = data_sem3[i,fs_ratio*q_offset[3]:fs_ratio*(q_offset[3]+time_window),:,:].copy()
    del data_sem3
    with h5py.File(dirs+'sem4.mat', 'r') as file:
        data_sem4 = file[val_name][:]
    data_sem4_copy = np.zeros((data_sem4.shape[0],time_window*fs_ratio,data_sem4.shape[2],data_sem4.shape[3]))
    for i in range(18):
        if body_nonbody_raw_answer[i+18] == 0:
            data_sem4_copy[i,:,:,:] = data_sem4[i,fs_ratio*q_offset[2]:fs_ratio*(q_offset[2]+time_window),:,:].copy()
        if body_nonbody_raw_answer[i+18] == 1:
            data_sem4_copy[i,:,:,:] = data_sem4[i,fs_ratio*q_offset[3]:fs_ratio*(q_offset[3]+time_window),:,:].copy()
    del data_sem4
    with h5py.File(dirs+'sem7.mat', 'r') as file:
        data_sem7 = file[val_name][:]
    data_sem7_copy = np.zeros((data_sem7.shape[0],time_window*fs_ratio,data_sem7.shape[2],data_sem7.shape[3]))
    for i in range(18):
        if body_nonbody_raw_answer[i+36] == 0:
            data_sem7_copy[i,:,:,:] = data_sem7[i,fs_ratio*q_offset[2]:fs_ratio*(q_offset[2]+time_window),:,:].copy()
        if body_nonbody_raw_answer[i+36] == 1:
            data_sem7_copy[i,:,:,:] = data_sem7[i,fs_ratio*q_offset[3]:fs_ratio*(q_offset[3]+time_window),:,:].copy()
    del data_sem7
    with h5py.File(dirs+'sem8.mat', 'r') as file:
        data_sem8 = file[val_name][:]
    data_sem8_copy = np.zeros((data_sem8.shape[0],time_window*fs_ratio,data_sem8.shape[2],data_sem8.shape[3]))
    for i in range(18):
        if body_nonbody_raw_answer[i+54] == 0:
            data_sem8_copy[i,:,:,:] = data_sem8[i,fs_ratio*q_offset[2]:fs_ratio*(q_offset[2]+time_window),:,:].copy()
        if body_nonbody_raw_answer[i+54] == 1:
            data_sem8_copy[i,:,:,:] = data_sem8[i,fs_ratio*q_offset[3]:fs_ratio*(q_offset[3]+time_window),:,:].copy()
    del data_sem8

    body_nonbody_raw_data = np.concatenate((data_sem3_copy, data_sem4_copy,data_sem7_copy,data_sem8_copy), axis=0)
    print("body_nonbody_load_complete. || shape : ",   body_nonbody_raw_data.shape)
    
    return liv_nonliv_raw_data, body_nonbody_raw_data

In [ ]:
#answer giving file
# liv category - 강아지 : puppy, 곰: bear, 사람 : haman,
# nonliv category - 눈사람 : snowman, 책 : book, 모자 : hat
# bodypart category - 코 : nose, 눈 : eye, 손 :hand
# nonbodypart category - 꽃 : flower, 총 : gun, 컵: cup
sem1_answers=['강아지','곰','눈사람','모자','강아지','책','곰','모자','사람','눈사람','곰','책','강아지','눈사람','사람','책','모자','사람']
sem2_answers=['사람','곰','모자','강아지','곰','눈사람','사람','책','모자','강아지','눈사람','책','강아지','책','눈사람','사람','곰','모자']
sem3_answers=['눈','손','꽃','컵','눈','총','손','컵','코','꽃','손','총','눈','꽃',"코","총","컵","코"]
sem4_answers=['코','손','컵','눈','손','꽃','코','총','컵','눈','꽃','총','눈','총',"꽃",'코','손','컵']
sem5_answers=['눈사람','사람','강아지','모자','눈사람','곰','책','강아지','사람','모자','곰','책','곰','모자','사람','강아지','눈사람','책']
sem6_answers=['책','강아지','사람','눈사람','모자','곰','강아지','곰','책','사람','곰','모자','책','눈사람','강아지','모자','사람','눈사람']
sem7_answers=['꽃','코','눈','컵','꽃','손','총','눈','코','컵','손','총','손','컵','코','눈','꽃','총']
sem8_answers=['총','눈','코','꽃','컵','손','눈','손','총','코','손','컵','총','꽃','눈','컵','코','꽃']



#binary giver
def liv_nonliv_answer_giver(answer_list):
    empty_return=[]
    for answer in answer_list:
        if answer in ['사람','곰','강아지']:
            empty_return.append(0)
        else: #모자, 눈사람, 책
            empty_return.append(1)
    np_return = np.array(empty_return)

    return np_return

def body_nonbody_answer_giver(answer_list):
    empty_return=[]
    for answer in answer_list:
        if answer in ['손','눈','코']:
            empty_return.append(0)
        else: #컵, 총, 꽃
            empty_return.append(1)
    np_return = np.array(empty_return)

    return np_return
    
liv_nonliv_raw_answer = liv_nonliv_answer_giver(sem1_answers+sem2_answers+sem5_answers+sem6_answers)

body_nonbody_raw_answer = body_nonbody_answer_giver(sem3_answers+sem4_answers+sem7_answers+sem8_answers)

def l_each_answer_list_giver(answer_list):
    anslist=[]
    for ansnum in range(len(answer_list)):
        if answer_list[ansnum] == "사람":
            anslist.append(0)
        if answer_list[ansnum] == "곰":
            anslist.append(1)
        if answer_list[ansnum] == "강아지":
            anslist.append(2)
        if answer_list[ansnum] == "모자":
            anslist.append(3)
        if answer_list[ansnum] == "눈사람":
            anslist.append(4)
        if answer_list[ansnum] == "책":
            anslist.append(5)
            
    np_return = np.array(anslist)
    
    return np_return 

        
def b_each_answer_list_giver(answer_list):
    anslist=[]
    for ansnum in range(len(answer_list)):
        if answer_list[ansnum] == "손":
            anslist.append(0)
        if answer_list[ansnum] == "눈":
            anslist.append(1)
        if answer_list[ansnum] == "코":
            anslist.append(2)
        if answer_list[ansnum] == "컵":
            anslist.append(3)
        if answer_list[ansnum] == "총":
            anslist.append(4)
        if answer_list[ansnum] == "꽃":
            anslist.append(5)
    np_return = np.array(anslist)
    
    return np_return 

l_each_raw_answer = l_each_answer_list_giver(sem1_answers+sem2_answers+sem5_answers+sem6_answers)
b_each_raw_answer = b_each_answer_list_giver(sem3_answers+sem4_answers+sem7_answers+sem8_answers)

In [ ]:
total_answer_not = np.concatenate((liv_nonliv_raw_answer,body_nonbody_raw_answer))
total_answer_sess = np.concatenate((np.zeros(72), np.ones(72)))
total_answer_4 = total_answer_not + total_answer_sess*2

In [ ]:
total_answer_4dim=np.zeros((144,4))
for i in range(144):
    total_answer_4dim[i,int(total_answer_4[i])] = 1

In [ ]:
total_answer_4dim.shape

In [ ]:
total_answer = np.concatenate((np.array([total_answer_not, total_answer_sess]).T, total_answer_4dim),axis=1)
print(total_answer.shape)

In [ ]:
def split_data(x_data, y_data, split_ratio=0.2, random_seed=None):
    if random_seed is not None:
        np.random.seed(random_seed)  
    
    indices = np.arange(x_data.shape[0])
    np.random.shuffle(indices)
    
    split_size = int(len(indices) * split_ratio)
    test_indices = indices[:split_size]
    train_indices = indices[split_size:]
    
    x_train = x_data[train_indices]
    y_train = y_data[train_indices]
    x_test = x_data[test_indices]
    y_test = y_data[test_indices]
    
    return x_train, y_train, x_test, y_test

In [ ]:
def four_paired_data_plotter(data1, data2, data3, data4, rdata1, rdata2, rdata3, rdata4, iter_num, pat_name,freq_str,titlename,ch,goodpval_list, mode="r2"):
    plt.plot(np.mean(data1,axis=1), label='(liv+body) vs (nonliv+nonbody)', c='b')
    plt.plot(np.mean(data2,axis=1), label ='(liv+nonliv) vs (body+nonbody) ', c='g')
    plt.plot(np.mean(data3,axis=1), label ='2 x 2 ', c='r')
    plt.plot(np.mean(data4,axis=1), label ='2 x 2, w.t. interaction', c='magenta')
    plt.plot(np.mean(rdata1,axis=1),  c='b', linestyle = "--")
    plt.plot(np.mean(rdata2,axis=1), c='g', linestyle = "--")
    plt.plot(np.mean(rdata3,axis=1),  c='r', linestyle = "--")
    plt.plot(np.mean(rdata4,axis=1),  c='magenta', linestyle = "--")
    tick_positions = np.arange(0, 48)
    freq_tick_value=[i*50 for i in range(1,49) ]
    title = titlename +" | "+pat_name +' ch'+str(ch)+' '+ freq_str
    plt.title(title ,fontsize=15)
    plt.ylabel(mode,fontsize=14)
    plt.xticks(ticks=tick_positions, labels=freq_tick_value,fontsize=13)
    plt.xlabel('Time (ms); center of window',fontsize=14)
    z_value = 1.96
    sem_data_1 = np.std(data1, axis=1) / np.sqrt(iter_num)
    sem_data_2 = np.std(data2, axis=1) / np.sqrt(iter_num)
    sem_data_3 = np.std(data3, axis=1) / np.sqrt(iter_num)
    sem_data_4 = np.std(data4, axis=1) / np.sqrt(iter_num)
    sem_rdata_1 = np.std(rdata1, axis=1) / np.sqrt(iter_num)
    sem_rdata_2 = np.std(rdata2, axis=1) / np.sqrt(iter_num)
    sem_rdata_3 = np.std(rdata3, axis=1) / np.sqrt(iter_num)
    sem_rdata_4 = np.std(rdata4, axis=1) / np.sqrt(iter_num)
    
    ci_lower_1 = np.mean(data1,axis=1) - z_value * sem_data_1
    ci_upper_1 = np.mean(data1,axis=1) + z_value * sem_data_1
    ci_lower_2 = np.mean(data2,axis=1) - z_value * sem_data_2
    ci_upper_2 = np.mean(data2,axis=1) + z_value * sem_data_2
    ci_lower_3 = np.mean(data3,axis=1) - z_value * sem_data_3
    ci_upper_3 = np.mean(data3,axis=1) + z_value * sem_data_3
    ci_lower_4 = np.mean(data4,axis=1) - z_value * sem_data_4
    ci_upper_4 = np.mean(data4,axis=1) + z_value * sem_data_4

    rci_lower_1 = np.mean(rdata1,axis=1) - z_value * sem_rdata_1
    rci_upper_1 = np.mean(rdata1,axis=1) + z_value * sem_rdata_1
    rci_lower_2 = np.mean(rdata2,axis=1) - z_value * sem_rdata_2
    rci_upper_2 = np.mean(rdata2,axis=1) + z_value * sem_rdata_2
    rci_lower_3 = np.mean(rdata3,axis=1) - z_value * sem_rdata_3
    rci_upper_3 = np.mean(rdata3,axis=1) + z_value * sem_rdata_3
    rci_lower_4 = np.mean(rdata4,axis=1) - z_value * sem_rdata_4
    rci_upper_4 = np.mean(rdata4,axis=1) + z_value * sem_rdata_4
    if mode == 'r':
        max_point = max(np.max(np.array([np.mean(data1,axis=1),np.mean(data2,axis=1),np.mean(data3,axis=1),np.mean(data4,axis=1)]) ),0.6) + 0.1
    else :
        max_point = max(np.max(np.array([np.mean(data1,axis=1),np.mean(data2,axis=1),np.mean(data3,axis=1),np.mean(data4,axis=1)]) ),0.4) + 0.1
    for t in range(11):
        if goodpval_list[0][t] == 1:
            plt.plot(t,  max_point - 0.04 , marker='*', color='b', markersize=3)
        if goodpval_list[1][t] == 1:
            plt.plot(t,  max_point - 0.03, marker='*', color='g', markersize=3 ) 
        if goodpval_list[2][t] == 1:
            plt.plot(t, max_point - 0.02, marker='*', color='r', markersize=3 ) 
        if goodpval_list[3][t] == 1:
            plt.plot(t, max_point - 0.01, marker='*', color='magenta', markersize=4) 
    plt.fill_between(range(len(data1)), ci_lower_1, ci_upper_1, color='b', alpha=0.2)
    plt.fill_between(range(len(data2)), ci_lower_2, ci_upper_2, color='g', alpha=0.2)
    plt.fill_between(range(len(data3)), ci_lower_3, ci_upper_3, color='r', alpha=0.2)
    plt.fill_between(range(len(data4)), ci_lower_4, ci_upper_4, color='magenta', alpha=0.2)
    plt.fill_between(range(len(rdata1)), rci_lower_1, rci_upper_1, color='b', alpha=0.1)
    plt.fill_between(range(len(rdata2)), rci_lower_2, rci_upper_2, color='g', alpha=0.1)
    plt.fill_between(range(len(rdata3)), rci_lower_3, rci_upper_3, color='r', alpha=0.1)
    plt.fill_between(range(len(rdata4)), rci_lower_4, rci_upper_4, color='magenta', alpha=0.1)
    if mode == 'r' :
        plt.ylim(-0.05,max_point)
    if mode == 'r2' :
        plt.ylim(-0.1,max_point)
    
    if titlename =="R2 score (train)":
        savepath="encoding_graph_TR_A/"+mode+"_train_"+pat_name +'_ch'+str(ch)+'_'+freq_str+'.png'
    else:
        savepath="encoding_graph_TR_/"+mode+"_"+pat_name +'_ch'+str(ch)+'_'+freq_str+'.png'
    plt.savefig(savepath)
    plt.close()

In [ ]:
def p_t_value(data1, data2):
    t_stat, p_value = ttest_ind(data1, data2, equal_var=False)  # 등분산성 가정 여부 설정
    p_value_one_sided = p_value / 2 if t_stat > 0 else 1 - (p_value / 2)
    return p_value_one_sided, t_stat

In [ ]:
def get_rank(a, b):
    sorted_b = sorted(b, reverse=True)  # b를 복사하여 정렬
    rank = 1
    for num in sorted_b:
        if a > num:
            break
        rank += 1
    return rank

In [ ]:
from scipy.stats import gaussian_kde
from scipy.integrate import trapz
def kde_fixed_h(samples, h_abs):
    sigma = np.array(samples).std(ddof=1)
    bw_factor = h_abs / sigma
    return gaussian_kde(samples, bw_method=bw_factor)

def get_KLdivergence(data1,ans1,data2,ans2,ans_num,itr_num):
    ans1_list = [[],[],[],[]]
    ans2_list = [[],[],[],[]]
    for i in range(len(ans1)):
        if ans1[i] == 0 :
            ans1_list[0].append(i)
        elif ans1[i] == 1 :
            ans1_list[1].append(i)
        elif ans1[i] == 2 :
            ans1_list[2].append(i)
        elif ans1[i] == 3 :
            ans1_list[3].append(i)
    for i in range(len(ans2)):
        if ans2[i] == 0 :
            ans2_list[0].append(i)
        elif ans2[i] == 1:
            ans2_list[1].append(i)
        elif ans2[i] == 2:
            ans2_list[2].append(i)
        elif ans2[i] == 3:
            ans2_list[3].append(i)
    data1_list = [data1[ans1_list[k]] for k in range(ans_num)]
    data2_list = [data2[ans2_list[k]] for k in range(ans_num)]
    all_samples = np.concatenate([data1, data2])
    h_global = 1.06 * all_samples.std(ddof=1) * len(all_samples) ** (-1/5)  # Silverman
    
    kde1_list = [kde_fixed_h(arr, h_global) for arr in data1_list]
    kde2_list = [kde_fixed_h(arr, h_global) for arr in data2_list]

    pi_t1   = np.array([len(ans1_list[i]) for i in range(ans_num)]) / np.sum(np.array([len(ans1_list[i]) for i in range(ans_num)]))
    pi_t2   = np.array([len(ans2_list[i]) for i in range(ans_num)]) / np.sum(np.array([len(ans2_list[i]) for i in range(ans_num)]))

    xmin_raw = np.min(all_samples)
    xmax_raw = np.max(all_samples)
    margin = 3 * h_global
    xmin, xmax = xmin_raw - margin, xmax_raw + margin
    grid = np.linspace(xmin, xmax, 500)
    
    eps = 1e-12                                # log(0) 방지용
    kl_total = 0.0

    for k in range(ans_num):
      for k_ in range(ans_num):
        kl_diff = 0 
        if k_ == k:
            p = kde1_list[k](grid) + eps             # f1ᵏ(x)
            q = kde2_list[k_](grid) + eps             # f2ᵏ(x)
            kl_k = trapz(p * np.log(p / q), grid)  # ∫ p log p/q dx
            kl_same = pi_t1[k] * (np.log(pi_t1[k]/pi_t2[k_]) + kl_k)
        else:
            p = kde1_list[k](grid) + eps             # f1ᵏ(x)
            q = kde2_list[k_](grid) + eps             # f2ᵏ(x)
            kl_k = trapz(p * np.log(p / q), grid)  # ∫ p log p/q dx
            kl_diff += pi_t1[k] * (np.log(pi_t1[k]/pi_t2[k_]) + kl_k)
        kl_total += kl_same - kl_diff / (ans_num -1)
    
    #print(f"delta KL = {kl_total:.6f}")
    randkl_list=[]
    for rand_itr in range(itr_num):
        ans2_shuffled = np.random.permutation(ans2)
        #print(ans2_shuffled)
        randans2_list = [[],[],[],[]]
        for i in range(len(ans2_shuffled)):
            if ans2_shuffled[i] == 0:
                randans2_list[0].append(i)
            if ans2_shuffled[i] == 1:
                randans2_list[1].append(i)
            if ans2_shuffled[i] == 2:
                randans2_list[2].append(i)
            if ans2_shuffled[i] == 3:
                randans2_list[3].append(i)
        #print(randans2_list)
        randdata2_list = [data2[randans2_list[k]] for k in range(ans_num)]
        randkde2_list = [kde_fixed_h(arr,h_global) for arr in randdata2_list]
        eps = 1e-12                                # log(0) 방지용
        kl_total_ = 0.0
        for k in range(ans_num):
          for k_ in range(ans_num):
            kl_diff_ = 0.0 
            if k == k_:
                p = kde1_list[k](grid) + eps             # f1ᵏ(x)
                q = randkde2_list[k_](grid) + eps             # f2ᵏ(x)
                kl_k = trapz(p * np.log(p / q), grid)  # ∫ p log p/q dx
                kl_same_= pi_t1[k] * (np.log(pi_t1[k]/pi_t2[k_]) + kl_k)
            else:
                p = kde1_list[k](grid) + eps             # f1ᵏ(x)
                q = randkde2_list[k_](grid) + eps             # f2ᵏ(x)
                kl_k = trapz(p * np.log(p / q), grid)  # ∫ p log p/q dx
                kl_diff_ += pi_t1[k] * (np.log(pi_t1[k]/pi_t2[k_]) + kl_k)
            kl_total_ += kl_same_ - kl_diff_ / (ans_num - 1)                       
        randkl_list.append(kl_total_)
    return kl_total, randkl_list

In [ ]:
with open('../topomap/left_bestmodel_index.pickle', 'rb') as f:
    left_bestmodel_index = pickle.load(f)

with open('../topomap/right_bestmodel_index.pickle', 'rb') as f:
    right_bestmodel_index = pickle.load(f)

In [ ]:
left_bestmodel_index

In [ ]:
from scipy.stats import percentileofscore

In [ ]:

from scipy.stats import percentileofscore
import random
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy.stats import ttest_ind
from sklearn.metrics import mean_squared_error, r2_score
total_subjects=['sub1','sub2','sub3','sub4','sub5','sub6','sub7','sub8','sub9','sub10','sub11','sub12','sub13','sub14','sub15']

lefthemi_subjects = ['sub1','sub2','sub3','sub4','sub5','sub6','sub10','sub13','sub14','sub15']
righthemi_subjects = ['sub3','sub7','sub8','sub9','sub11','sub12']
lefthemi_subjects_startnum = [0,47,113,114,177,233,313,325,373,401]

righthemi_subjects_startnum = [0,51,75,116,139,154]

ver=11
itr_num = 50000 
np.random.seed(42)  # 고정된 랜덤 시드 설정
random_seeds = np.random.randint(0, 50000, size=itr_num)
old_threshold = 0.05 / 11 / 624 / 2
new_threshold = 0.05 / 48 / 624 / 2
# 첫 10개 확인
tot_timecnt = 48
freq_name = ['theta','alpha','beta','LG1','LG2','HG']
for pat_name in total_subjects:
    print(pat_name)
            
    r_old_path = "save_dir/"+pat_name+"_corr_r.pkl"
    with open(r_old_path, 'rb') as f:
        pat_r_data_old_raw = pickle.load(f)
    pat_r_data_old = np.mean(pat_r_data_old_raw,axis=5)
    print(pat_r_data_old.shape)
    
    r_old_path = "save_dir/"+pat_name+"_r2score_test.pkl"
    with open(r_old_path, 'rb') as f:
        pat_r2_data_old_raw = pickle.load(f)
    pat_r2_data_old = np.mean(pat_r2_data_old_raw,axis=5)
    print(pat_r_data_old.shape)    
    
    p_old_path = "save_dir/"+pat_name+"_p_and_t.pkl"
    with open(p_old_path, 'rb') as f:
        pat_p_data_old_raw = pickle.load(f)
    pat_p_data_old = pat_p_data_old_raw[0][1,:,:,:,:]
    print(pat_p_data_old.shape)

    coef_old_path = "save_dir/"+pat_name+"_r_coef.pkl"
    with open(coef_old_path, 'rb') as f:
        pat_r_coef = pickle.load(f)
        
    
    coef_old_rand_path = "save_dir/"+pat_name+"_r_random_coef.pkl"
    with open(coef_old_rand_path, 'rb') as f:
        pat_r_random_coef = pickle.load(f)
    pat_r_random_coef = pat_r_random_coef
    
    
    bad_trial_for_chs = bad_trial_giver(pat_name,strip_grid_no_list)
    error_trial = liv_nonliv_badnull_ans_dict[pat_name][0]+liv_nonliv_badnull_ans_dict[pat_name][1]+body_nonbody_badnull_ans_dict[pat_name][0]+body_nonbody_badnull_ans_dict[pat_name][1]
    l_data_TR_A_raw, b_data_TR_A_raw = file_loader_and_npconcate_TFmean(pat_name,8000,10400,ver)
    l_data_Q, b_data_Q = file_loader_and_npconcate_question_TFmean(pat_name,q_offsettiming,liv_nonliv_raw_answer,body_nonbody_raw_answer,ver)
    
    print(l_data_Q.shape)
    l_data_Q_raw = l_data_Q[:,1100:,:,:]
    b_data_Q_raw = b_data_Q[:,1100:,:,:]
    l_data_TR_A = np.concatenate((l_data_Q_raw,l_data_TR_A_raw),axis=1)
    b_data_TR_A = np.concatenate((b_data_Q_raw,b_data_TR_A_raw),axis=1)
    raw_data_Q = np.concatenate((l_data_Q,b_data_Q))
    good_ch = getPatgoodch(pat_name, strip_grid_no_list)
    raw_data_TR_A = np.concatenate((l_data_TR_A, b_data_TR_A))
    raw_data_TR_A = raw_data_TR_A[:,:,good_ch,:]
    pat_random_kl = np.zeros((len(good_ch),itr_num))
    pat_kl = np.zeros((len(good_ch),tot_timecnt))
    type_index=[]

    if pat_name != 'sub3':
        if pat_name in lefthemi_subjects:
            for ch in range(len(good_ch)):
                sub_index = lefthemi_subjects.index(pat_name)
                if ch + lefthemi_subjects_startnum[sub_index] in left_bestmodel_index[0]:
                    type_index.append(0)
                elif ch + lefthemi_subjects_startnum[sub_index] in left_bestmodel_index[1]:
                    type_index.append(1)
                elif ch + lefthemi_subjects_startnum[sub_index] in left_bestmodel_index[2]:
                    type_index.append(2)
                elif ch + lefthemi_subjects_startnum[sub_index] in left_bestmodel_index[3]:
                    type_index.append(3)
        if pat_name in righthemi_subjects:
            for ch in range(len(good_ch)):
                sub_index = righthemi_subjects.index(pat_name)
                if ch + righthemi_subjects_startnum[sub_index] in right_bestmodel_index[0]:
                    type_index.append(0)
                elif ch + righthemi_subjects_startnum[sub_index] in right_bestmodel_index[1]:
                    type_index.append(1)
                elif ch + righthemi_subjects_startnum[sub_index] in right_bestmodel_index[2]:
                    type_index.append(2)
                elif ch + righthemi_subjects_startnum[sub_index] in right_bestmodel_index[3]:
                    type_index.append(3)
    elif pat_name == "sub3": #sub3 has electrodes both on right and left hemi
        for ch in range(len(good_ch)):
            if ch < 40:
                sub_index = righthemi_subjects.index(pat_name)
                if ch + righthemi_subjects_startnum[sub_index] in right_bestmodel_index[0]:
                    type_index.append(0)
                elif ch + righthemi_subjects_startnum[sub_index] in right_bestmodel_index[1]:
                    type_index.append(1)
                elif ch + righthemi_subjects_startnum[sub_index] in right_bestmodel_index[2]:
                    type_index.append(2)
                elif ch + righthemi_subjects_startnum[sub_index] in right_bestmodel_index[3]:
                    type_index.append(3)
            elif ch in [40,41]:
                sub_index = lefthemi_subjects.index(pat_name)
                if ch + lefthemi_subjects_startnum[sub_index] in left_bestmodel_index[0]:
                    type_index.append(0)
                elif ch + lefthemi_subjects_startnum[sub_index] in left_bestmodel_index[1]:
                    type_index.append(1)
                elif ch + lefthemi_subjects_startnum[sub_index] in left_bestmodel_index[2]:
                    type_index.append(2)
                elif ch + lefthemi_subjects_startnum[sub_index] in left_bestmodel_index[3]:
                    type_index.append(3)
            else:
                sub_index = righthemi_subjects.index(pat_name)
                if ch + righthemi_subjects_startnum[sub_index]+2 in right_bestmodel_index[0]:
                    type_index.append(0)
                elif ch + righthemi_subjects_startnum[sub_index]+2 in right_bestmodel_index[1]:
                    type_index.append(1)
                elif ch + righthemi_subjects_startnum[sub_index]+2 in right_bestmodel_index[2]:
                    type_index.append(2)
                elif ch + righthemi_subjects_startnum[sub_index]+2 in right_bestmodel_index[3]:
                    type_index.append(3)
                    
    total_old_max_r_index = np.zeros((6,len(good_ch),4))
    for freq in range(5,4,-1): #only for HG
        print("================",freq_name[freq],"================")
        
        for ch in range(len(good_ch)):
            print("|",end='')
            bad_trial_total_time = [[] for i in range(tot_timecnt)]
            for bins_ in range(-1,6):
                if bins_ ==-1:
                    t_start = 0
                    t_end = 1
                else:
                    t_start = 8*bins_
                    t_end = min(48,8*bins_+9)            
                for t_cnt in range(t_start,t_end):
                    bad_trial_total_time[t_cnt] += bad_trial_for_chs[bins_+1][ch]
            
            each_r_data_old = pat_r_data_old[0,:,freq,ch,:]
            each_p_data_old = pat_p_data_old[:,freq,ch,:]
            each_p_data_cutted_old = np.where(each_p_data_old<old_threshold,1,0)
            old_max_r_index = []
            for it in range(4):
                if np.sum(each_p_data_cutted_old[it,:]) < 1 :
                    old_max_r_index.append(-1)
                else:
                    p_filtered = each_r_data_old[it,:] * each_p_data_cutted_old[it,:]
                    max_r_point_ = np.argmax(p_filtered)
                    old_max_r_index.append(max_r_point_)
            total_old_max_r_index[freq,ch,:] = np.array(old_max_r_index)
            activity_total_list = []
            for t in range(tot_timecnt):
                raw_data_TR_t = raw_data_TR_A[:, 100*t : 100*t +200 , : , :]
                data_TR_t = np.mean(raw_data_TR_t,axis=1)
                bad_trial_TR = bad_trial_total_time[t]
                good_trial_TR = [i for i in range(144) if i not in bad_trial_TR+error_trial]
                bad_trial_Q = bad_trial_for_chs[0][ch]
                good_trial_Q = [i for i in range(144) if i not in bad_trial_Q+error_trial]
                data_good_TR_t = data_TR_t[:,ch,freq][good_trial_Q].tolist()
                activity_total_list += data_good_TR_t

                for types in range(4):
                  if types == type_index[ch]:
                    if old_max_r_index[types] != -1:
                        data1_ = raw_data_Q[:,old_max_r_index[types]*100:old_max_r_index[types]*100+200,:,:]
                        data1_ = np.mean(data1_,axis=1)
                        if types == 0 :
                            tot_answer = total_answer_not
                            ans_num = 2
                        if types == 1 :
                            tot_answer = total_answer_sess
                            ans_num = 2
                        if types in [2,3]:
                            tot_answer = total_answer_4
                            ans_num = 4
                        
                        data1 = data1_[:,ch,freq][good_trial_Q]
                        ans1 = tot_answer[good_trial_Q]
                        data2 = data_TR_t[:,ch,freq][good_trial_TR]
                        ans2 = tot_answer[good_trial_TR]
                        kl, randkl_list = get_KLdivergence(data1,ans1,data2,ans2,ans_num,itr_num)
                        pat_kl[ch, t] = kl
                        pat_random_kl[ch,t,:] = np.array(randkl_list)
                        print(percentileofscore(pat_random_kl[ch,t,:], pat_kl[ch, t], kind="weak"), end=' ')
        save_file_name = pat_name + "_kldevergence_diff.pkl"
        with open(save_file_name, 'wb') as f:
            pickle.dump([pat_kl,pat_random_kl], f, pickle.HIGHEST_PROTOCOL)
